In [1]:
from collections import defaultdict
import numpy as np

MovieLens 100K Dataset : https://grouplens.org/datasets/movielens/

In [2]:
with open('ml-100k/u.data','r') as f:
    for line in f.readlines()[:10]:
        print(line)

196	242	3	881250949

186	302	3	891717742

22	377	1	878887116

244	51	2	880606923

166	346	1	886397596

298	474	4	884182806

115	265	2	881171488

253	465	5	891628467

305	451	3	886324817

6	86	3	883603013



In [3]:
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [4]:
class movieDataset(Dataset):
    def __init__(self, file, split):
        
        self.user_likes = defaultdict(set) # import collections
        self.user_dislikes = defaultdict(set)
        self.max_u_id = -1
        self.max_i_id = -1

        with open(file,'r') as f:
            for line in f.readlines():
                u,i,r,_ = line.split("\t")
                u = int(u)
                i = int(i)
                r = int(r)
                
                if r >= 3:
                    self.user_likes[u].add(i)
                else:
                    self.user_dislikes[u].add(i)
                    
                self.max_u_id = max(u,self.max_u_id)
                self.max_i_id = max(i,self.max_i_id)
            f.close()
        
    def __getitem__(self, index):
        u = torch.LongTensor(np.array([index]))
        
        temp=[]
        for item in self.user_likes[index]:
            temp.append(item)
        i = torch.LongTensor(np.array([temp]))
        
        temp=[]
        for item in self.user_dislikes[index]:
            temp.append(item)
        j = torch.LongTensor(np.array([temp]))
        
        return u, i, j

    def __len__(self):

        return self.max_u_id

    def max_value(self):
        return self.max_u_id,self.max_i_id

In [5]:
trainset = movieDataset(file='ml-100k/u.data', split='train')
x,y,z = trainset.__getitem__(196)
print(x.size())
print(y.size())
print(z.size())

torch.Size([1])
torch.Size([1, 34])
torch.Size([1, 5])


In [6]:
max_u_id,max_i_id = trainset.max_value()
print(max_u_id, max_i_id)
print(trainset.__len__())

943 1682
943


In [7]:
train_loader = DataLoader(dataset=trainset, batch_size=2, shuffle=False)

Args:
    
    u(torch.LongTensor): tensor stored user indexes. [batch_size]
    
    i(torch.LongTensor): tensor stored item indexes which is prefered by user. [batch_size,]
    
    j(torch.LongTensor): tensor stored item indexes which is not prefered by user. [batch_size,]

Returns:
    
    torch.FloatTensor : Return loss value.
 

In [8]:
class BPR(nn.Module):
    def __init__(self, user_size, item_size, dim, weight_decay):
        super().__init__()
        
        self.W = nn.Parameter(torch.empty(user_size, dim))
        self.H = nn.Parameter(torch.empty(item_size, dim))
        nn.init.xavier_normal_(self.W.data)
        nn.init.xavier_normal_(self.H.data)
        self.weight_decay = weight_decay

    def forward(self, u, i, j):
        
        u = self.W[u, :]
        i = self.H[i, :]
        j = self.H[j, :]
        x_ui = torch.mul(u, i).sum(dim=1)
        x_uj = torch.mul(u, j).sum(dim=1)
        x_uij = x_ui - x_uj
        log_prob = F.logsigmoid(x_uij).sum()
        regularization = self.weight_decay * (u.norm(dim=1).pow(2).sum() + i.norm(dim=1).pow(2).sum() + j.norm(dim=1).pow(2).sum())
        return -log_prob + regularization

    def recommend(self, u):
        
        u = self.W[u, :]
        x_ui = torch.mm(u, self.H.t())
        pred = torch.argsort(x_ui, dim=1)
        return pred


In [9]:
model = BPR(max_u_id, max_i_id, dim = 30, weight_decay = 0.01).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.02)

In [10]:
print(model)

BPR()


In [11]:
# from torchviz import make_dot
# batch = next(iter(train_loader))
# yhat = model(batch.text)
# make_dot(yhat, params=dict(list(model.named_parameters()))).render("test_BPR", format="png")

In [12]:
u,i,j = trainset.__getitem__(196)
W = nn.Parameter(torch.empty(1682, 30))
W.size()

torch.Size([1682, 30])

In [13]:
i = W[i, :]
i
i.size()

torch.Size([1, 34, 30])

In [14]:
j = W[j, :]
j
j.size()

torch.Size([1, 5, 30])

In [15]:
u,i,j = trainset.__getitem__(196)
optimizer.zero_grad()
loss = model(u, i, j)
print(loss)
loss.backward()
optimizer.step()

tensor(20.7940, device='cuda:0', grad_fn=<AddBackward0>)


In [21]:
for x in range(1) : # locky number
    for y in range(900):
        u,i,j = trainset.__getitem__(y)
        optimizer.zero_grad()
        loss = model(u, i, j)
        print(x, y, loss)
        loss.backward()
        optimizer.step()
        
    u,i,j = trainset.__getitem__(196)
    model.recommend(u)

0 0 tensor(20.8366, device='cuda:0', grad_fn=<AddBackward0>)
0 1 tensor(92.2810, device='cuda:0', grad_fn=<AddBackward0>)
0 2 tensor(31.3219, device='cuda:0', grad_fn=<AddBackward0>)
0 3 tensor(41.5426, device='cuda:0', grad_fn=<AddBackward0>)
0 4 tensor(23.6310, device='cuda:0', grad_fn=<AddBackward0>)
0 5 tensor(54.4013, device='cuda:0', grad_fn=<AddBackward0>)
0 6 tensor(41.8753, device='cuda:0', grad_fn=<AddBackward0>)
0 7 tensor(93.4186, device='cuda:0', grad_fn=<AddBackward0>)
0 8 tensor(29.4735, device='cuda:0', grad_fn=<AddBackward0>)
0 9 tensor(22.6946, device='cuda:0', grad_fn=<AddBackward0>)
0 10 tensor(51.7159, device='cuda:0', grad_fn=<AddBackward0>)
0 11 tensor(47.0924, device='cuda:0', grad_fn=<AddBackward0>)
0 12 tensor(23.8666, device='cuda:0', grad_fn=<AddBackward0>)
0 13 tensor(104.4182, device='cuda:0', grad_fn=<AddBackward0>)
0 14 tensor(34.9257, device='cuda:0', grad_fn=<AddBackward0>)
0 15 tensor(33.7317, device='cuda:0', grad_fn=<AddBackward0>)
0 16 tensor(53.62

0 132 tensor(10.9637, device='cuda:0', grad_fn=<AddBackward0>)
0 133 tensor(19.8545, device='cuda:0', grad_fn=<AddBackward0>)
0 134 tensor(19.6345, device='cuda:0', grad_fn=<AddBackward0>)
0 135 tensor(12.5246, device='cuda:0', grad_fn=<AddBackward0>)
0 136 tensor(17.0575, device='cuda:0', grad_fn=<AddBackward0>)
0 137 tensor(22.3062, device='cuda:0', grad_fn=<AddBackward0>)
0 138 tensor(28.1964, device='cuda:0', grad_fn=<AddBackward0>)
0 139 tensor(14.9892, device='cuda:0', grad_fn=<AddBackward0>)
0 140 tensor(15.0766, device='cuda:0', grad_fn=<AddBackward0>)
0 141 tensor(33.0070, device='cuda:0', grad_fn=<AddBackward0>)
0 142 tensor(16.1354, device='cuda:0', grad_fn=<AddBackward0>)
0 143 tensor(14.8330, device='cuda:0', grad_fn=<AddBackward0>)
0 144 tensor(33.8266, device='cuda:0', grad_fn=<AddBackward0>)
0 145 tensor(84.0402, device='cuda:0', grad_fn=<AddBackward0>)
0 146 tensor(18.8901, device='cuda:0', grad_fn=<AddBackward0>)
0 147 tensor(24.5145, device='cuda:0', grad_fn=<AddBack

0 266 tensor(20.0303, device='cuda:0', grad_fn=<AddBackward0>)
0 267 tensor(65.7231, device='cuda:0', grad_fn=<AddBackward0>)
0 268 tensor(85.0344, device='cuda:0', grad_fn=<AddBackward0>)
0 269 tensor(61.2844, device='cuda:0', grad_fn=<AddBackward0>)
0 270 tensor(34.3137, device='cuda:0', grad_fn=<AddBackward0>)
0 271 tensor(75.4450, device='cuda:0', grad_fn=<AddBackward0>)
0 272 tensor(42.6677, device='cuda:0', grad_fn=<AddBackward0>)
0 273 tensor(16.2844, device='cuda:0', grad_fn=<AddBackward0>)
0 274 tensor(44.2202, device='cuda:0', grad_fn=<AddBackward0>)
0 275 tensor(52.0673, device='cuda:0', grad_fn=<AddBackward0>)
0 276 tensor(185.2596, device='cuda:0', grad_fn=<AddBackward0>)
0 277 tensor(26.1634, device='cuda:0', grad_fn=<AddBackward0>)
0 278 tensor(18.5290, device='cuda:0', grad_fn=<AddBackward0>)
0 279 tensor(172.2822, device='cuda:0', grad_fn=<AddBackward0>)
0 280 tensor(66.9784, device='cuda:0', grad_fn=<AddBackward0>)
0 281 tensor(15.0862, device='cuda:0', grad_fn=<AddBa

0 400 tensor(12.1255, device='cuda:0', grad_fn=<AddBackward0>)
0 401 tensor(54.7662, device='cuda:0', grad_fn=<AddBackward0>)
0 402 tensor(48.0979, device='cuda:0', grad_fn=<AddBackward0>)
0 403 tensor(22.6459, device='cuda:0', grad_fn=<AddBackward0>)
0 404 tensor(23.7171, device='cuda:0', grad_fn=<AddBackward0>)
0 405 tensor(277.1342, device='cuda:0', grad_fn=<AddBackward0>)
0 406 tensor(111.2634, device='cuda:0', grad_fn=<AddBackward0>)
0 407 tensor(109.9062, device='cuda:0', grad_fn=<AddBackward0>)
0 408 tensor(18.9265, device='cuda:0', grad_fn=<AddBackward0>)
0 409 tensor(73.2271, device='cuda:0', grad_fn=<AddBackward0>)
0 410 tensor(15.8484, device='cuda:0', grad_fn=<AddBackward0>)
0 411 tensor(52.6238, device='cuda:0', grad_fn=<AddBackward0>)
0 412 tensor(42.9823, device='cuda:0', grad_fn=<AddBackward0>)
0 413 tensor(28.6467, device='cuda:0', grad_fn=<AddBackward0>)
0 414 tensor(19.4236, device='cuda:0', grad_fn=<AddBackward0>)
0 415 tensor(23.5271, device='cuda:0', grad_fn=<AddB

0 547 tensor(12.5211, device='cuda:0', grad_fn=<AddBackward0>)
0 548 tensor(25.0123, device='cuda:0', grad_fn=<AddBackward0>)
0 549 tensor(15.1173, device='cuda:0', grad_fn=<AddBackward0>)
0 550 tensor(12.1281, device='cuda:0', grad_fn=<AddBackward0>)
0 551 tensor(48.9069, device='cuda:0', grad_fn=<AddBackward0>)
0 552 tensor(22.9344, device='cuda:0', grad_fn=<AddBackward0>)
0 553 tensor(31.9548, device='cuda:0', grad_fn=<AddBackward0>)
0 554 tensor(21.4019, device='cuda:0', grad_fn=<AddBackward0>)
0 555 tensor(19.8264, device='cuda:0', grad_fn=<AddBackward0>)
0 556 tensor(26.1130, device='cuda:0', grad_fn=<AddBackward0>)
0 557 tensor(18.0840, device='cuda:0', grad_fn=<AddBackward0>)
0 558 tensor(32.3558, device='cuda:0', grad_fn=<AddBackward0>)
0 559 tensor(22.4619, device='cuda:0', grad_fn=<AddBackward0>)
0 560 tensor(21.0802, device='cuda:0', grad_fn=<AddBackward0>)
0 561 tensor(107.5226, device='cuda:0', grad_fn=<AddBackward0>)
0 562 tensor(17.6005, device='cuda:0', grad_fn=<AddBac

0 684 tensor(22.2308, device='cuda:0', grad_fn=<AddBackward0>)
0 685 tensor(20.6043, device='cuda:0', grad_fn=<AddBackward0>)
0 686 tensor(21.8727, device='cuda:0', grad_fn=<AddBackward0>)
0 687 tensor(12.0915, device='cuda:0', grad_fn=<AddBackward0>)
0 688 tensor(11.8878, device='cuda:0', grad_fn=<AddBackward0>)
0 689 tensor(14.9548, device='cuda:0', grad_fn=<AddBackward0>)
0 690 tensor(42.2990, device='cuda:0', grad_fn=<AddBackward0>)
0 691 tensor(14.8583, device='cuda:0', grad_fn=<AddBackward0>)
0 692 tensor(18.2465, device='cuda:0', grad_fn=<AddBackward0>)
0 693 tensor(44.0478, device='cuda:0', grad_fn=<AddBackward0>)
0 694 tensor(52.7063, device='cuda:0', grad_fn=<AddBackward0>)
0 695 tensor(21.5977, device='cuda:0', grad_fn=<AddBackward0>)
0 696 tensor(18.5496, device='cuda:0', grad_fn=<AddBackward0>)
0 697 tensor(23.5782, device='cuda:0', grad_fn=<AddBackward0>)
0 698 tensor(31.7912, device='cuda:0', grad_fn=<AddBackward0>)
0 699 tensor(26.2414, device='cuda:0', grad_fn=<AddBack

0 820 tensor(16.0174, device='cuda:0', grad_fn=<AddBackward0>)
0 821 tensor(15.7107, device='cuda:0', grad_fn=<AddBackward0>)
0 822 tensor(16.6142, device='cuda:0', grad_fn=<AddBackward0>)
0 823 tensor(26.6292, device='cuda:0', grad_fn=<AddBackward0>)
0 824 tensor(16.9986, device='cuda:0', grad_fn=<AddBackward0>)
0 825 tensor(33.2366, device='cuda:0', grad_fn=<AddBackward0>)
0 826 tensor(34.0091, device='cuda:0', grad_fn=<AddBackward0>)
0 827 tensor(11.0042, device='cuda:0', grad_fn=<AddBackward0>)
0 828 tensor(25.6528, device='cuda:0', grad_fn=<AddBackward0>)
0 829 tensor(24.6381, device='cuda:0', grad_fn=<AddBackward0>)
0 830 tensor(37.3400, device='cuda:0', grad_fn=<AddBackward0>)
0 831 tensor(19.5601, device='cuda:0', grad_fn=<AddBackward0>)
0 832 tensor(16.7262, device='cuda:0', grad_fn=<AddBackward0>)
0 833 tensor(52.6563, device='cuda:0', grad_fn=<AddBackward0>)
0 834 tensor(19.9815, device='cuda:0', grad_fn=<AddBackward0>)
0 835 tensor(27.6025, device='cuda:0', grad_fn=<AddBack

In [22]:
u,i,j = trainset.__getitem__(196)
model.recommend(u)[0][:10]

tensor([1247,  839,  437,  439, 1192,  377, 1091, 1003, 1522, 1093],
       device='cuda:0')